In [1]:
import numpy as np
import mygrad
import mygrad.functional as F
import mygrad.nn as nn

In [2]:
from sklearn.datasets import fetch_california_housing

X,y = fetch_california_housing(return_X_y=True)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
X_train.shape

(15480, 8)

In [5]:
class FCNet(nn.Module):
    def __init__(self, name, init="xavier_normal"):
        super(FCNet, self).__init__(name)
        
        #Specifying activation calculates function-specific gain for initialization
        self.linear1 = nn.Linear(f"{name}.linear1", 8, 32, init=init, activation="Tanh")
        self.tanh1 = F.tanh()
        self.linear2 = nn.Linear(f"{name}.linear2", 32, 1, init=init)
        
    def forward(self, X):
        X = self.linear1.forward(X)
        X = self.tanh1.forward(X)
        X = self.linear2.forward(X)
        
        return X
    
    def parameters(self, ):
        return self.linear1.parameters() + self.linear2.parameters()

In [6]:
history = []

model = FCNet("fcnet", init="he_normal") # best init for the task. Also try "xavier_normal", "xavier_uniform"

loss_fn = F.mse_loss()
optim = nn.RMSProp(model.parameters()) #Also try nn.Adam and nn.SGD

for i in range(500):
    ix = np.random.randint(0, len(X_train), 256)
    x_batch = mygrad.value(X_train[ix], "x_batch")
    y_batch = mygrad.value(y_train[ix], "y_batch")
    
    y = model.forward(x_batch)
    loss = loss_fn.forward(y, y_batch)
    
    loss_fn.backward()   #Slightly different from torch
    optim.step()
    
    optim.zero_grad()
    
    history.append(loss.data)

    if i % 10 == 0:
        print("step #%i | mean loss = %.3f" % (i, np.mean(history[-10:])))
    
    

Creating Function of type <class 'mygrad.functional.function.Matmul'> with name matmul_0
Creating Function of type <class 'mygrad.functional.function.Add'> with name add_0
Invalid mode: n_in. Using n_in
Creating Function of type <class 'mygrad.functional.function.Tanh'> with name tanh_0
Creating Function of type <class 'mygrad.functional.function.Matmul'> with name matmul_1
Creating Function of type <class 'mygrad.functional.function.Add'> with name add_1
Invalid mode: n_in. Using n_in
Creating Function of type <class 'mygrad.functional.function.MSELoss'> with name mse_loss_0
step #0 | mean loss = 6.193
step #10 | mean loss = 1.988
step #20 | mean loss = 0.591
step #30 | mean loss = 0.521
step #40 | mean loss = 0.582
step #50 | mean loss = 0.516
step #60 | mean loss = 0.488
step #70 | mean loss = 0.506
step #80 | mean loss = 0.467
step #90 | mean loss = 0.506
step #100 | mean loss = 0.486
step #110 | mean loss = 0.467
step #120 | mean loss = 0.433
step #130 | mean loss = 0.430
step #14

C:\Users\Aren\Documents\mygrad\mygrad\functional\function.py:201: RuntimeWarning: overflow encountered in exp
  t = 2 / (1 + np.exp(-2 * x.data)) - 1


In [7]:
X_train = mygrad.value(X_train, "X_train", requires_grad=False)
X_test = mygrad.value(X_test, "X_test", requires_grad=False)
y_train = mygrad.value(y_train, "y_train", requires_grad=False)
y_test = mygrad.value(y_test, "y_test", requires_grad=False)

y_train_pred = model.forward(X_train)
y_test_pred = model.forward(X_test)

train_loss = loss_fn.forward(y_train_pred, y_train)
test_loss = loss_fn.forward(y_test_pred, y_test)

print(f"Train MSE: {train_loss.data}")
print(f"Test MSE: {test_loss.data}")

Train MSE: 0.38138099701705874
Test MSE: 0.3807619217668671
